# Skull AI

The objective here is to make AI/Bots that can play the deception board/tile game: Skull. See this link [LINK] to a video showing how to play and then the flow diagram in the gitrepo showing the decision flow for  a player.

ASSUMPTIONS:
 - Only 4 players in a game
 - for now, wont pay attention to previous rounds
 - simple terms (linear)
 - max bet to place is 12

### Library imports

In [1]:
import random
from numpy.random import rand, choice
import numpy as np
import pandas as pd

### Player and Game classes

Player class.
Decisions for the bot to make:
 - What tile to place first [DONE]
 - To bet or place another tile
     - binary classifications
 - What tile to place [DONE]
     - binary classification
 - what to bet [DONE]
     - multiclass classification/ integer prediction
 - to bet or pass [DONE]
     - binary classification
 - who to pick when seeking
     - multiclass classification
 
Defining a class for the player:
 - a player needs to be able to place a flower and skull [DONE]
 - a player needs to be able to remove a tile
     - randomly if they're skulled by somone else [DONE]
     - choose if they skull themselves
 - 

In [89]:
class skullPlayer:
    def __init__(self):
        self.flowers=3
        self.skull=1
        self.totalTiles=4
        self.out=False
        self.hasSkull=True
        self.placedTiles=[]
        self.point=0
        self.theta_tp=(rand(1,30)*2)-1
        self.theta_bp=(rand(1,31)*2)-1
        self.theta_w2b=(rand(12,95)*2-1)
        self.theta_bpa=(rand(1,95)*2-1)
        self.theta_w2p=(rand(3,73)*2-1)
        self.hasPassed=False
        self.bet=0
        
    def summary(self): #TESTED
        print("Flowers: {}\nSkull: {}\nPoint: {}\nPlaced tiles: {}".format(self.flowers,self.skull,self.point, self.placedTiles))
    
    def placeFlower(self): #TESTED
        if len(self.placedTiles)<self.totalTiles:
            if (len(self.placedTiles)-sum(self.placedTiles))<self.flowers:
                self.placedTiles.append(False)
                
    def placeSkull(self): #TESTED
        if len(self.placedTiles)<self.totalTiles:
            if self.hasSkull & (sum(self.placedTiles)==0):
                self.placedTiles.append(True)
                
    def reset(self): #TESTED
        self.placedTiles=[]
    
    def fullReset(self):
        self.flowers=3
        self.skull=1
        self.totalTiles=4
        self.out=False
        self.hasSkull=True
        self.placedTiles=[]
        self.point=0
        self.hasPassed=False
        self.bet=0
                
    def removeRandom(self): #TESTED
        if self.hasSkull:
            if choice([True]+[False]*self.flowers):
                self.skull+=-1
                self.hasSkull=False
            else:
                self.flowers+=-1
        else:
            self.flowers+=-1
        self.totalTiles+=-1
        if self.totalTiles==0:
            self.out=True
        self.reset()
    
    def chooseTileToPlace(self,externalInfo): #TESTED
        """
        Function to decide what tile to place.
        External info should have for each player:
        - If they have a point (binary)
        - How many tiles they have (numerical)
        - How many tiles they have placed (numerical)
        - If they have their skull (dual-binary) [1,0] skull, [0,0] unknown, [0,1] no skull 
        - their place in the turn order (numerical)
        - if they're out of the game (binary) [1] out, [0] in
        - optional (not implemented)
            - if they just lost a tile
            - if they just won a point
        LAST row should be that on the "bot"
        Extra
        - random float for "randomness"
        - bias term
        therefore external info is a 1x30 vector
        
        RETURNS
        boolean. true, place a skull, false place a flower
        """
        
        inVec=np.reshape(externalInfo,(-1,1))
        inVec=np.concatenate((inVec,np.array([[rand()],[1]])))
        
        skull = ((1/(1+np.exp(-np.dot(self.theta_tp,inVec))))>=0.5)[0][0]
        
        if skull:
            self.placeSkull()
        else:
            self.placeFlower()
        
        return skull
    
    def placeTile(self,externalInfo): #TESTED
        if (any(self.placedTiles)|(not(self.hasSkull))):
            self.placeFlower()
        elif not(any(self.placedTiles)) & len(self.placedTiles)==self.flowers:
            self.placeSkull()
        else:
            self.chooseTileToPlace(externalInfo)
    
    def betOrPlace(self,externalInfo,betRec): #TESTED
        """
        Function to decide to start the betting or place another tile
        Same input variables as choosing what to place and 
        -include if they have placed a skull already (binary) [1] have placed a skull
        EXTRA
        - random float for "randomness"
        - bais term
        
        RETURNS
        boolean. true, make a bet, false, place a tile
        """
        if len(self.placedTiles)<self.totalTiles:
            inVec=np.reshape(y.externalInfo,(-1,1))
            inVec=np.concatenate((inVec,np.array([[int(any(self.placedTiles))],[rand()],[1]])))

            betOrPlace = ((1/(1+np.exp(-np.dot(self.theta_bp,inVec))))>=0.5)[0][0]

            if betOrPlace:
                bet = self.whatToBet(externalInfo,betRec)
                print("Player bets: {}".format(bet))
                self.bet=bet
                return bet
            else:
                self.placeTile(externalInfo)
                print("Tile placed")
                return "PLACE"
        else:
            bet = self.whatToBet(externalInfo,betRec)
            print("Player bets: {}".format(bet))
            return bet
            
            
    def whatToBet(self,externalInfo,betRec): #TESTED
        """
        Function to decide what to bet
        Uses the board state record and the current betRecord. Also uses if they player has placed a skull, random float, bias and a list of available bets
        """
        inVec=np.concatenate((np.reshape(externalInfo,(-1,1)),np.reshape(betRec,(-1,1)),np.array([[int(any(self.placedTiles))],[rand()],[1]])))
        availableBets=np.zeros((1,12))
        availableBets[:,0:sum(externalInfo[:,2])]=1
        cb=(np.where(betRec[:,:-1].any(axis=0))[0])
        if cb.shape[0]==0:
            cb=0
        else:
            cb=max(cb)+1
        availableBets[:,0:cb]=0
        inVec=np.concatenate((inVec,availableBets.T))
        bet=np.argmax((1/(1+np.exp(-np.dot(self.theta_w2b,inVec))))*availableBets.T)+1
        self.bet=bet
        return bet
    
    def betOrPass(self,externalInfo,betRec): #TESTED
        """
        Function to decide to bet or pass
        Same input variables as choosing what to place and 
        -include if they have placed a skull already (binary) [1] have placed a skull
        EXTRA
        - random float for "randomness"
        - bais term
        
        RETURNS
        boolean. true, make a bet, false, place a tile
        """
        inVec=np.concatenate((np.reshape(externalInfo,(-1,1)),np.reshape(betRec,(-1,1)),np.array([[int(any(self.placedTiles))],[rand()],[1]])))
        availableBets=np.zeros((1,12))
        availableBets[:,0:sum(externalInfo[:,2])]=1
        cb=(np.where(betRec.any(axis=0))[0])
        if cb.shape[0]==0:
            cb=0
        else:
            cb=max(cb)+1
        availableBets[:,0:cb]=0
        inVec=np.concatenate((inVec,availableBets.T))
        betOrPass=(1/(1+np.exp(-np.dot(self.theta_bpa,inVec))))>=0.5
        
        if betOrPass:
            bet = self.whatToBet(externalInfo,betRec)
            print("Player bets: {}".format(bet))
            return bet
        else:
            self.hasPassed=True
            print("Player passes")
            return "PASS"
        
    def whoToPick(self,externalInfo,betRec,pickrec): #TESTED
        """
        Function to decide whose top tile to turn over when i pick
        uses board state, bet rec (does not need self information for these)
        and self pick rec
        extra
         - num of tiles to go
         - bias
        """
        inVec=np.concatenate((np.reshape(externalInfo,(-1,1)),np.reshape(betRec,(-1,1)),np.reshape(pickrec,(-1,1)),np.array([[1]])))
        whoToPick = np.argmax((1/(1+np.exp(-np.dot(self.theta_w2p,inVec))))*pickrec[:,-1].reshape(3,1))
        return whoToPick
        

Game glass

In [100]:
class skullGame:
    def __init__(self,player1,player2,player3,player4):
        self.playerArray = [player1,player2,player3,player4]
        self.firstPlayerOfRound = random.randint(0,len(self.playerArray)-1)
        self.externalInfo = np.array([[0,4,0,1,0,((i-self.firstPlayerOfRound)%4),0] for i in range(0,4)])
        self.initPlacement()
        self.betRec=np.zeros((len(self.playerArray),(len(self.playerArray)*3)+1))
        self.pickRec=np.zeros((len(self.playerArray)-1,4))
        self.pickRec[:,-1]=1
        
    def initPlacement(self):
        for ind in range(0,len(self.playerArray)):
            if not(self.playerArray[ind].out):
                self.playerArray[ind].placeTile(self.externalInfo[list(range(0,ind))+list(range(ind+1,len(self.playerArray)))+[ind],:])
                self.externalInfo[ind][2]=1
            
    def playerSkulled(self,seekingPlayerInd,skulledByInd=-1):
        if skulledByInd==-1:
            print("Player has skulled themselves")
        else:
            print("Player skulled by player {}".format(skulledByInd))
            self.externalInfo[skulledByInd,3]=1
        self.playerArray[seekingPlayerInd].removeRandom()
        self.externalInfo[seekingPlayerInd,1]+=-1
        self.externalInfo[seekingPlayerInd,3]=0
        if currentPlayer.out:
            self.externalInfo[seekingPlayerInd,-1]=1
            self.externalInfo[seekingPlayerInd][2]=0
            print("Player eliminated")
        self.resetPlayers(seekingPlayerInd)
            
    def resetPlayers(self,seekingPlayerInd):
        for players in self.playerArray:
            players.reset()
        self.initPlacement()
        self.setFirstPlayerIndex(seekingPlayerInd)
        self.betRec=np.zeros((len(self.playerArray),(len(self.playerArray)*3)+1))
        self.pickRec=np.zeros((len(self.playerArray)-1,4))
        self.pickRec[:,-1]=1
        self.setTurnOrders()
        
    def setTurnOrders(self):
        place=0
        for i in range(0,4):
            if not(self.playerArray[(self.firstPlayerOfRound+i)%4].out):
                self.externalInfo[(self.firstPlayerOfRound+i)%4,5]=place
                place+=1
            else:
                self.externalInfo[(self.firstPlayerOfRound+i)%4,5]=4
            self.playerArray[(self.firstPlayerOfRound+i)%4].hasPassed=False
            
    def setFirstPlayerIndex(self,seekingPlayerInd):
        for i in range(0,4):
            if not(self.playerArray[(seekingPlayerInd+i)%4].out):
                self.firstPlayerOfRound=(seekingPlayerInd+i)%4
                break
                
    def main(self): #SOMETHING BROKEN IN THIS?!
        roundCount=0
        while True:                                                                                        #Begin the game
            i=self.firstPlayerOfRound                                                                               #get the first player of the round
            roundCount+=1                                                                                        #increase the round count by 1
            print("\nRound: {}".format(roundCount))                                                                #print the current round count
            turnCount=0                                                                                          #init the turn count for the round
            currentbet=0                                                                                         #init the current bet tracker
            print("Board State:\n {}".format(self.externalInfo))                                                   #print the current board state
            while True:                                                                                          #while in the betting/placing stage
                turnCount+=1                                                                                     #increase the turn count by 1
                print("Turn: {}".format(turnCount))                                                              #print the turn count
                for x in range(0,4):                                                                             #loop through the players
                    z=(x+i)%4                                                                                    #go to the player who is first in the round 
                    print("player"+str(z))                                                                       #print who the current player is
                    currentPlayer=self.playerArray[z]                                                               #placeholder var for current player
                    if currentPlayer.out:                                                                        #if the current player is out of the game, they "pass" 
                        print("Player {} is out".format(z))                                                     #print the statement
                        self.betRec[z,-1]=1                                                                         #note it in the bet rec
                        continue                                                                                 #move onto the next player
                    if currentPlayer.hasPassed:                                                                  #if the player has already passed
                        print("Player {} has already passed".format(z))                                          #print the statement
                        continue                                                                                 #move onto the next loop
                    dataOrder = list(range(0,z))+list(range(z+1,len(self.playerArray)))+[z]                         #sort the order to present the data
                    if not(self.betRec.any()):                                                                      #if there has been no bets so far
                        ans = currentPlayer.betOrPlace(self.externalInfo[dataOrder,:],self.betRec[dataOrder,:])        #decide to bet or place
                        if (type(ans)==np.int64):                                                                #if they bet
                            self.betRec[z,ans-1]=1                                                                  #note it in the bec recs
                            currentbet=ans                                                                       #assign the current bet
                            print(y.betRec)                                                                      #print the bet record at this point                 
                        elif (type(ans)==str):                                                                   #if they place a tile
                            self.externalInfo[z,2]+=1                                                               #increase it in the external info
                            print(currentPlayer.placedTiles)                                                    #print what tiles the player has down
                            print("Board State:\n {}".format(self.externalInfo))                                   #print the current board state
                        else:                                                                                    #if an unexpected output is recieved
                            raise ansError("output type error from betOrPlace")                                  #throw error
                    else:                                                                                        #if there are bets
                        if not(currentPlayer.hasPassed):                                                         #if the current player hasn't passed
                            ans = currentPlayer.betOrPass(self.externalInfo[dataOrder,:],self.betRec[dataOrder,:])     #decide to bet or pass
                            if type(ans)==np.int64:                                                              #if the player bets
                                self.betRec[z,ans-1]=1                                                              #note it in the bec recs
                                currentbet=ans                                                                   #assign the current bet
                                print(self.betRec)                                                                  #print the bet record at this point 
                            elif (type(ans)==str):                                                               #if the player passes
                                self.betRec[z,-1]=1                                                                 #note the pass
                            else:                                                                                #if an unexpected output is recieved
                                raise ansError("output type error from betOrPass")                               #throw error
                        else:                                                                                    #if player has passed
                            print("Player {} has already passed".format(z))                                      #print passed
                            continue                                                                             #continue to next player
                    if (sum(self.betRec[:,-1])==(len(self.playerArray)-1))|(currentbet==sum(self.externalInfo[:,2]))|(currentbet==12):     #if all have passed or max bet made
                        break                                                                                    #stop looping through players
                if (sum(self.betRec[:,-1])==(len(self.playerArray)-1)):                                                #if other remaining players have passed
                    print("All players passed")                                                                  #print that statement
                    z=np.where(1-(self.betRec[:,-1]))[0][0]                                                         #get the index of the player thats going to be seeking
                    currentPlayer=self.playerArray[z]                                                               #set the placeholder var
                    dataOrder = list(range(0,z))+list(range(z+1,len(self.playerArray)))+[z]                         #set the order to present the data
                    print("Player to seek: {}".format(z))                                                        #print which player is actually seeking
                    break                                                                                        #break out of the betting/placing while loop
                elif(currentbet==sum(self.externalInfo[:,2])):                                                      #if the max bet was made
                    print("Max bet made")                                                                        #print that statement
                    print("Player to seek: {}".format(z))                                                        #print which player is seeking
                    break                                                                                        #break out of the betting/placing loop
                if turnCount>24:                                                                                 #if the round has lasted longer than 24 turns then something has gone wrong
                    raise loopError("was stuck in inf loop")                                                     #raise an error
            print("Player's tiles: {}".format(currentPlayer.placedTiles))                                       #print the seeking players tiles
            if any(currentPlayer.placedTiles):                                                                   #if they have played a skull
                self.playerSkulled(z)                                                                               #remove tile and reset round
            else:                                                                                                #if they have not skulled themselves
                toFind=currentPlayer.bet                                                                         #establish how many they need to find
                print("Total flowers to find: {}".format(toFind))                                               #print the goal
                found=len(currentPlayer.placedTiles)                                                             #set how many they have
                print("Flowers found: {}".format(found))                                                        #print it
                self.pickRec[:,1]=self.externalInfo[dataOrder[0:-1],2]                                                 #in the pick rec set how many tiles other have to pick from
                self.pickRec[:,-1]=(1-self.externalInfo[dataOrder[0:-1],-1])                                           #any players that are out set as ileligible
                while found<toFind:                                                                              #whilst the player does not have the required number of flowers
                    print("Remaining: {}".format(toFind-found))                                                 #print how many they need to find
                    pickInd=currentPlayer.whoToPick(self.externalInfo[dataOrder[0:-1]],self.betRec[dataOrder[0:-1]],self.pickRec) #get the player to pick someone
                    pickPlayer=dataOrder[pickInd]                                                                #transform that into player id
                    print("Picks player: {}".format(pickPlayer))                                                #print who they have picked
                    tile=self.playerArray[pickPlayer].placedTiles.pop()                                             #pop the last tile from the player they have picked
                    if tile:                                                                                     #if it's a skull
                        self.playerSkulled(z,pickPlayer)                                                            #remove tile, start new round etc.
                        break                                                                                    #and break out of while loop
                    else:                                                                                        #if it was a flower
                        print("Flower found")                                                                   #print statement
                        found+=1                                                                                 #increase found count
                        print("Flowers found: {}".format(found))
                        self.pickRec[pickInd,0]=1                                                                   #note that the player has been picked before
                        self.pickRec[pickInd,1]+=-1                                                                 #decrease the number of eligible tiles
                        self.pickRec[pickInd,2]+=1                                                                  #increase the number of tile picked from this person
                        if self.pickRec[pickInd,1]==0:                                                              #if the player has no tiles remaining
                            self.pickRec[pickInd,3]=0                                                               #set ileligibility
                            if self.pickRec[pickInd,2]==self.externalInfo[pickPlayer,1]:                               #if the number of tiles picked from player is all they have with no skull
                                self.externalInfo[pickPlayer,4]=1                                                   #it means they have no skull left for the game, note it
                        print(self.pickRec)                                                                        #print the bet rec
                if found==toFind:                                                                                #if required number has been found
                    print("All Flowers found")                                                                  #print the statement
                    print("Player {} gets 1 point\n".format(z))                                                 #print what player gets a point
                    self.externalInfo[z,0]+=1                                                                       #note it in the external info
                    self.resetPlayers(z)                                                                            #reset the players
                    if self.externalInfo[z,0]==2:                                                                   #if the player now has 2 points
                        print("Player {} wins the game!".format(z))                                              #theyve won the game
                        return self.playerArray[z]
                        break                                                                                    #break the loop as game is over
                else:                                                                                            #REDUNDANT
                    self.resetPlayers(z)                                                                            #REDUNDANT
            if sum(self.externalInfo[:,-1])==3:                                                                     #if all but one player is elinimiated 
                print("All other players eliminated")                                                           #print statement
                print("Player {} wins the game!".format(np.argmin(self.externalInfo[:,-1])))                        #the remaining player has won the game!
                return self.playerArray[np.argmin(self.externalInfo[:,-1])]
                break                                                                                            #break game loop


## Testing zone

In [101]:
playerList=[skullPlayer() for x in range(0,4)]
y=skullGame(*playerList)
winner = y.main()
winner.summary()


Round: 1
Board State:
 [[0 4 1 1 0 1 0]
 [0 4 1 1 0 2 0]
 [0 4 1 1 0 3 0]
 [0 4 1 1 0 0 0]]
Turn: 1
player3
Player bets: 3
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
player0
Player passes
player1
Player passes
player2
Player passes
All players passed
Player to seek: 3
Player's tiles: [True]
Player has skulled themselves
Player eliminated

Round: 2
Board State:
 [[0 4 1 1 0 1 0]
 [0 4 1 1 0 2 0]
 [0 4 1 1 0 3 0]
 [0 3 1 0 0 0 1]]
Turn: 1
player3
Player bets: 3
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
player0
Player bets: 4
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Max bet made
Player to seek: 0
Player's tiles: [

In [94]:
y.externalInfo

array([[0, 0, 0, 0, 0, 4, 1],
       [0, 3, 1, 0, 0, 0, 1],
       [0, 4, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 4, 1]])

In [35]:
for player in playerList:
    player.summary()

Flowers: 3
Skull: 1
Point: 0
Placed tiles: [True]
Flowers: 3
Skull: 1
Point: 0
Placed tiles: [True]
Flowers: 3
Skull: 1
Point: 0
Placed tiles: [True]
Flowers: 3
Skull: 1
Point: 0
Placed tiles: [True]


In [36]:
pd.DataFrame(y.externalInfo, 
             index=["player1","player2","player3","player4"],
             columns=["has_point","tiles","tiles_placed","skull_has","skull_none","turn_order","out"])

,has_point,tiles,tiles_placed,skull_has,skull_none,turn_order,out
player1,0,4,1,1,0,3,0
player2,0,4,1,1,0,0,0
player3,0,4,1,1,0,1,0
player4,0,4,1,1,0,2,0


In [37]:
pd.DataFrame(y.betRec,index=["player1","player2","player3","player4"],columns=[str(i) for i in range(1,13)]+["PASS"])

,1,2,3,4,5,6,7,8,9,10,11,12,PASS
player1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
player2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
player3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
player4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
pd.DataFrame(y.pickRec,index=["Player"+str(i) for i in range(1,4)],columns=["prevPicked?","tiles_remaining","tiles_selected","eligible to pick"])

,prevPicked?,tiles_remaining,tiles_selected,eligible to pick
Player1,0.0,0.0,0.0,1.0
Player2,0.0,0.0,0.0,1.0
Player3,0.0,0.0,0.0,1.0


In [104]:
playerList=[skullPlayer() for x in range(0,4)]
y=skullGame(*playerList)
gameActive=True


roundCount=0
while gameActive:                                                                                        #Begin the game
    i=y.firstPlayerOfRound                                                                               #get the first player of the round
    roundCount+=1                                                                                        #increase the round count by 1
    print("\nRound: {}".format(roundCount))                                                                #print the current round count
    turnCount=0                                                                                          #init the turn count for the round
    currentbet=0                                                                                         #init the current bet tracker
    print("Board State:\n {}".format(y.externalInfo))                                                   #print the current board state
    while True:                                                                                          #while in the betting/placing stage
        turnCount+=1                                                                                     #increase the turn count by 1
        print("Turn: {}".format(turnCount))                                                              #print the turn count
        for x in range(0,4):                                                                             #loop through the players
            z=(x+i)%4                                                                                    #go to the player who is first in the round 
            print("player"+str(z))                                                                       #print who the current player is
            currentPlayer=y.playerArray[z]                                                               #placeholder var for current player
            if currentPlayer.out:                                                                        #if the current player is out of the game, they "pass" 
                print("Player {} is out".format(z))                                                     #print the statement
                y.betRec[z,-1]=1                                                                         #note it in the bet rec
                continue                                                                                 #move onto the next player
            if currentPlayer.hasPassed:                                                                  #if the player has already passed
                print("Player {} has already passed".format(z))                                          #print the statement
                continue                                                                                 #move onto the next loop
            dataOrder = list(range(0,z))+list(range(z+1,len(y.playerArray)))+[z]                         #sort the order to present the data
            if not(y.betRec.any()):                                                                      #if there has been no bets so far
                ans = currentPlayer.betOrPlace(y.externalInfo[dataOrder,:],y.betRec[dataOrder,:])        #decide to bet or place
                if (type(ans)==np.int64):                                                                #if they bet
                    y.betRec[z,ans-1]=1                                                                  #note it in the bec recs
                    currentbet=ans                                                                       #assign the current bet
                    print(y.betRec)                                                                      #print the bet record at this point                 
                elif (type(ans)==str):                                                                   #if they place a tile
                    y.externalInfo[z,2]+=1                                                               #increase it in the external info
                    print(currentPlayer.placedTiles)                                                    #print what tiles the player has down
                    print("Board State:\n {}".format(y.externalInfo))                                   #print the current board state
                else:                                                                                    #if an unexpected output is recieved
                    raise ansError("output type error from betOrPlace")                                  #throw error
            else:                                                                                        #if there are bets
                if not(currentPlayer.hasPassed):                                                         #if the current player hasn't passed
                    ans = currentPlayer.betOrPass(y.externalInfo[dataOrder,:],y.betRec[dataOrder,:])     #decide to bet or pass
                    if type(ans)==np.int64:                                                              #if the player bets
                        y.betRec[z,ans-1]=1                                                              #note it in the bec recs
                        currentbet=ans                                                                   #assign the current bet
                        print(y.betRec)                                                                  #print the bet record at this point 
                    elif (type(ans)==str):                                                               #if the player passes
                        y.betRec[z,-1]=1                                                                 #note the pass
                    else:                                                                                #if an unexpected output is recieved
                        raise ansError("output type error from betOrPass")                               #throw error
                else:                                                                                    #if player has passed
                    print("Player {} has already passed".format(z))                                      #print passed
                    continue                                                                             #continue to next player
            if (sum(y.betRec[:,-1])==(len(y.playerArray)-1))|(currentbet==sum(y.externalInfo[:,2]))|(currentbet==12):     #if all have passed or max bet made
                break                                                                                    #stop looping through players
        if (sum(y.betRec[:,-1])==(len(y.playerArray)-1)):                                                #if other remaining players have passed
            print("All players passed")                                                                  #print that statement
            z=np.where(1-(y.betRec[:,-1]))[0][0]                                                         #get the index of the player thats going to be seeking
            currentPlayer=y.playerArray[z]                                                               #set the placeholder var
            dataOrder = list(range(0,z))+list(range(z+1,len(y.playerArray)))+[z]                         #set the order to present the data
            print("Player to seek: {}".format(z))                                                        #print which player is actually seeking
            break                                                                                        #break out of the betting/placing while loop
        elif(currentbet==sum(y.externalInfo[:,2])):                                                      #if the max bet was made
            print("Max bet made")                                                                        #print that statement
            print("Player to seek: {}".format(z))                                                        #print which player is seeking
            break                                                                                        #break out of the betting/placing loop
        if turnCount>24:                                                                                 #if the round has lasted longer than 24 turns then something has gone wrong
            raise loopError("was stuck in inf loop")                                                     #raise an error
    print("Player's tiles: {}".format(currentPlayer.placedTiles))                                       #print the seeking players tiles
    if any(currentPlayer.placedTiles):                                                                   #if they have played a skull
        y.playerSkulled(z)                                                                               #remove tile and reset round
    else:                                                                                                #if they have not skulled themselves
        toFind=currentPlayer.bet                                                                         #establish how many they need to find
        print("Total flowers to find: {}".format(toFind))                                               #print the goal
        found=len(currentPlayer.placedTiles)                                                             #set how many they have
        print("Flowers found: {}".format(found))                                                        #print it
        y.pickRec[:,1]=y.externalInfo[dataOrder[0:-1],2]                                                 #in the pick rec set how many tiles other have to pick from
        y.pickRec[:,-1]=(1-y.externalInfo[dataOrder[0:-1],-1])                                           #any players that are out set as ileligible
        while found<toFind:                                                                              #whilst the player does not have the required number of flowers
            print("Remaining: {}".format(toFind-found))                                                 #print how many they need to find
            pickInd=currentPlayer.whoToPick(y.externalInfo[dataOrder[0:-1]],y.betRec[dataOrder[0:-1]],y.pickRec) #get the player to pick someone
            pickPlayer=dataOrder[pickInd]                                                                #transform that into player id
            print("Picks player: {}".format(pickPlayer))                                                #print who they have picked
            tile=y.playerArray[pickPlayer].placedTiles.pop()                                             #pop the last tile from the player they have picked
            if tile:                                                                                     #if it's a skull
                y.playerSkulled(z,pickPlayer)                                                            #remove tile, start new round etc.
                break                                                                                    #and break out of while loop
            else:                                                                                        #if it was a flower
                print("Flower found")                                                                   #print statement
                found+=1                                                                                 #increase found count
                print("Flowers found: {}".format(found))
                y.pickRec[pickInd,0]=1                                                                   #note that the player has been picked before
                y.pickRec[pickInd,1]+=-1                                                                 #decrease the number of eligible tiles
                y.pickRec[pickInd,2]+=1                                                                  #increase the number of tile picked from this person
                if y.pickRec[pickInd,1]==0:                                                              #if the player has no tiles remaining
                    y.pickRec[pickInd,3]=0                                                               #set ileligibility
                    if y.pickRec[pickInd,2]==y.externalInfo[pickPlayer,1]:                               #if the number of tiles picked from player is all they have with no skull
                        y.externalInfo[pickPlayer,4]=1                                                   #it means they have no skull left for the game, note it
                print(y.pickRec)                                                                        #print the bet rec
        if found==toFind:                                                                                #if required number has been found
            print("All Flowers found")                                                                  #print the statement
            print("Player {} gets 1 point\n".format(z))                                                 #print what player gets a point
            y.externalInfo[z,0]+=1                                                                       #note it in the external info
            y.resetPlayers(z)                                                                            #reset the players
            if y.externalInfo[z,0]==2:                                                                   #if the player now has 2 points
                print("Player {} wins the game!".format(z))                                              #theyve won the game
                break                                                                                    #break the loop as game is over
        else:                                                                                            #REDUNDANT
            y.resetPlayers(z)                                                                            #REDUNDANT
    if sum(y.externalInfo[:,-1])==3:                                                                     #if all but one player is elinimiated 
        print("All other players eliminated")                                                           #print statement
        print("Player {} wins the game!".format(np.argmin(y.externalInfo[:,-1])))                        #the remaining player has won the game!
        break                                                                                            #break game loop
        


Round: 1
Board State:
 [[0 4 1 1 0 1 0]
 [0 4 1 1 0 2 0]
 [0 4 1 1 0 3 0]
 [0 4 1 1 0 0 0]]
Turn: 1
player3
Tile placed
[True, False]
Board State:
 [[0 4 1 1 0 1 0]
 [0 4 1 1 0 2 0]
 [0 4 1 1 0 3 0]
 [0 4 2 1 0 0 0]]
player0
Tile placed
[True, False]
Board State:
 [[0 4 2 1 0 1 0]
 [0 4 1 1 0 2 0]
 [0 4 1 1 0 3 0]
 [0 4 2 1 0 0 0]]
player1
Player bets: 4
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
player2
Player passes
Turn: 2
player3
Player bets: 5
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
player0
Player passes
player1
Player bets: 6
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Max bet made
Player to seek: 1
Player

0.0

3